# Tutorial 4
In this tutorial, we are going to train a small Convolutional Neural Network using TensorFlow and convert it to an SNN using the few-spike encoding scheme.

The ANN and converted SNN both achieve around 99% on the MNIST test set.
## Install PyGeNN wheel from Google Drive
Download wheel file

In [1]:
!gdown 1fllqUtL_1_tyGzjNHSR-9i5fXFI9jqAi 

Downloading...
From: https://drive.google.com/uc?id=1fllqUtL_1_tyGzjNHSR-9i5fXFI9jqAi
To: /content/pygenn-4.8.1-cp310-cp310-linux_x86_64.whl
100% 22.3M/22.3M [00:00<00:00, 203MB/s]


and then install PyGeNN from wheel file

In [2]:
!pip install pygenn-4.8.1-cp310-cp310-linux_x86_64.whl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./pygenn-4.8.1-cp310-cp310-linux_x86_64.whl


and checkout mlGeNN from git and install boththe ``ml_genn`` and ``ml_genn_tf`` packages

In [3]:
!git clone https://github.com/genn-team/ml_genn.git
%pushd ml_genn
!git checkout 983400ba
!pip install ./ml_genn
!pip install ./ml_genn_tf
%popd

Cloning into 'ml_genn'...
remote: Enumerating objects: 6790, done.
remote: Counting objects: 100% (2597/2597), done.
remote: Compressing objects: 100% (988/988), done.
remote: Total 6790 (delta 1613), reused 2484 (delta 1582), pack-reused 4193
Receiving objects: 100% (6790/6790), 37.13 MiB | 15.86 MiB/s, done.
Resolving deltas: 100% (4468/4468), done.
/content/ml_genn
Note: switching to '983400ba'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at 983400b softmax is required for eprop validation split
Looking in 

Set environment variable to allow GeNN to find CUDA

In [4]:
%env CUDA_PATH=/usr/local/cuda

env: CUDA_PATH=/usr/local/cuda


## Train ANN
Firstly we define a simple ANN in Keras with two convolutional layers followed by two dense layers and train it:

In [5]:
from tensorflow.keras import models, layers, datasets
from tensorflow.config import experimental

# Irritatingly, TF's default GPU memory allocator  allocates 
# all available GPU memory - this can't be freed and would leave
# none for mlGeNN so we turn off this behaviour
for gpu in experimental.list_physical_devices("GPU"):
    experimental.set_memory_growth(gpu, True)

# Load MNIST data and normalise to [0,1]
(train_x, train_y), (test_x, test_y) = datasets.mnist.load_data()
train_x = train_x.reshape((-1, 28, 28, 1)) / 255.0
test_x = test_x.reshape((-1, 28, 28, 1)) / 255.0

# Create and compile TF model
tf_model = models.Sequential([
    layers.Conv2D(16, 5, padding="valid", activation="relu", use_bias=False, input_shape=train_x.shape[1:]),
    layers.AveragePooling2D(2),
    layers.Conv2D(8, 5, padding="valid", activation="relu", use_bias=False),
    layers.AveragePooling2D(2),
    layers.Flatten(),
    layers.Dense(128, activation="relu", use_bias=False),
    layers.Dense(64, activation="relu", use_bias=False),
    layers.Dense(train_y.max() + 1, activation="softmax", use_bias=False),
], name="simple_cnn")
tf_model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Fit TF model
tf_model.fit(train_x, train_y, epochs=10)

11490434/11490434 [==============================] - 2s 0us/step
Epoch 1/10
1875/1875 [==============================] - 16s 3ms/step - loss: 0.2230 - accuracy: 0.9331
Epoch 2/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0781 - accuracy: 0.9760
Epoch 3/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0603 - accuracy: 0.9809
Epoch 4/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0474 - accuracy: 0.9853
Epoch 5/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0408 - accuracy: 0.9872
Epoch 6/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0331 - accuracy: 0.9892
Epoch 7/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0295 - accuracy: 0.9905
Epoch 8/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0249 - accuracy: 0.9919
Epoch 9/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0220 - accuracy: 0.9928
Epoch 10/10
1875

## Evaluate ANN model
Now we evaluate the ANN on the MNIST test set:

In [6]:
tf_model.evaluate(test_x, test_y)

313/313 [==============================] - 1s 4ms/step - loss: 0.0381 - accuracy: 0.9893


[0.03811901435256004, 0.989300012588501]

## Build normalization dataset
To correctly configure the conversion algorithm, the range of activations in each layer is required. We determine this from a single, randomly selected batch of training data. Slightly awkwardly, mlGeNN takes these as an iterator so we turn them into a TF dataset:

In [7]:
import numpy as np
from tensorflow.data import Dataset, AUTOTUNE

# ML GeNN norm dataset
norm_i = np.random.choice(train_x.shape[0], 128, replace=False)

norm_ds = Dataset.from_tensor_slices((train_x[norm_i], train_y[norm_i]))
norm_ds = norm_ds.batch(128)
norm_ds = norm_ds.prefetch(AUTOTUNE)

## Convert model
We are going to use the few-spike conversion scheme to convert the ANN to an SNN with $k=8$ timesteps per examples:

Stöckl, Christoph, and Wolfgang Maass. 2021. “Optimized Spiking Neurons Can Classify Images with High Accuracy through Temporal Coding with Two Spikes.” Nature Machine Intelligence 3(3): 230–38 ([doi](http://dx.doi.org/10.1038/s42256-021-00311-4))


In [8]:
from ml_genn_tf.converters import FewSpike

# Build few-spike converter
converter = FewSpike(k=8, norm_data=[norm_ds])

# Convert and compile ML GeNN model
net, net_inputs, net_outputs, tf_layer_pops = converter.convert(tf_model)

## Compilation
In mlGeNN, in order to turn an abstract network description into something that can actually be used for training or inference you use a *compiler* class. Here, we ask the converter to build us a suitable compiler and specify batch size and that we don't want connectvity expanded into sparse connectivity.

In [9]:
compiler = converter.create_compiler(prefer_in_memory_connect=False, batch_size=128)
compiled_net = compiler.compile(net, inputs=net_inputs, outputs=net_outputs)

## Evaluate SNN models
Finally, we evaluate the SNN model on the MNIST test set:

In [10]:
with compiled_net:
    compiled_net.evaluate({net_inputs[0]: test_x},
                          {net_outputs[0]: test_y})

  0%|          | 0/84 [00:00<?, ?it/s]